In [1]:

import os
import yaml
import time
import optuna

from datasets import load_dataset, load_from_disk
from plaid.bridges.huggingface_bridge import huggingface_dataset_to_plaid, huggingface_description_to_problem_definition
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.base import clone
from sklearn.model_selection import KFold

from ml_pipeline_nodes import PLAIDTransformer, ScalarScalerNode, GPRegressorNode, PCAEmbeddingNode, DatasetTargetTransformerRegressor
from sklearn.utils import estimator_html_repr

import numpy as np

import warnings
warnings.filterwarnings('ignore', module='sklearn')

/home/fabien/miniconda3/envs/plaid-dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("config.yml") as f:
    config = yaml.safe_load(f)

global_params = config["global"]

start = time.time()
hf_dataset = load_dataset(global_params['dataset_path'], split="all_samples")
# hf_dataset = load_from_disk(global_params['dataset_path'])
print(f"Loading dataset from HuggingFace Hub took: {time.time() - start:.2g} seconds")

prob_def = huggingface_description_to_problem_definition(hf_dataset.description)

train_split = prob_def.get_split(global_params['train_split_name'])[:24]
dataset_train, _ = huggingface_dataset_to_plaid(hf_dataset, ids = train_split, processes_number = 24)#os.cpu_count())

test_split = prob_def.get_split(global_params['test_split_name'])[:24]
dataset_test, _ = huggingface_dataset_to_plaid(hf_dataset, ids = test_split, processes_number = 24)#os.cpu_count())

del hf_dataset

Loading dataset from HuggingFace Hub took: 2.3 seconds
Converting huggingface dataset to plaid dataset...


100%|██████████| 24/24 [00:01<00:00, 19.43it/s]


Converting huggingface dataset to plaid dataset...


100%|██████████| 24/24 [00:00<00:00, 33.38it/s]


In [3]:
pipeline = Pipeline([
    ('test_0', PLAIDTransformer(features_param = config['test_0']['feature'])),
    ('test_1', PLAIDTransformer(features_param = config['test_1']['feature'])),
])
pipeline

,steps,"[('test_0', ...), ('test_1', ...)]"
,transform_input,None
,memory,None
,verbose,False
,features_param,"{'name': 'angle_in', 'type': 'scalar'}"
,features_param,"{'base_name': 'Base_2_2', 'name': 'mach', 'type': 'field'}"


In [7]:
test = PLAIDTransformer(features_param = config['test_0']['feature'])
print(test.features_param)
scalars = test.get_features(dataset_train)
print(scalars)

{'type': 'scalar', 'name': 'angle_in'}
{0: 45.5066666666667, 1: 47.8, 2: 46.68, 3: 23.08, 4: 45.72, 5: 34.9733333333333, 6: 34.1466666666667, 7: 23.96, 8: 22.3066666666667, 9: 35.7733333333333, 10: 23.8, 11: 34.3066666666667, 12: 45.6133333333333, 13: 29.4, 14: 29.0533333333333, 15: 29.2666666666667, 16: 11.2666666666667, 17: 15.5333333333333, 18: 16.3333333333333, 19: 15.0, 20: 38.2533333333333, 21: 16.9733333333333, 22: 16.5733333333333, 23: 48.3866666666667}


In [ ]:
scalars_doubled = {id:2*scalar for id, scalar in scalars.items()}
print(scalars_doubled)
print(dataset_train.get_sample_ids())
test.set_features(dataset_train, scalars_doubled)
print(scalars)

{0: 91.0133333333334, 1: 95.6, 2: 93.36, 3: 46.16, 4: 91.44, 5: 69.9466666666666, 6: 68.2933333333334, 7: 47.92, 8: 44.6133333333334, 9: 71.5466666666666, 10: 47.6, 11: 68.6133333333334, 12: 91.2266666666666, 13: 58.8, 14: 58.1066666666666, 15: 58.5333333333334, 16: 22.5333333333334, 17: 31.0666666666666, 18: 32.6666666666666, 19: 30.0, 20: 76.5066666666666, 21: 33.9466666666666, 22: 33.1466666666666, 23: 96.7733333333334}


AssertionError: Must provide the same sample indices in feature_identifiers and features as in the dataset